In [16]:
from q1_3 import get_stop_words
from q1_1 import Parse_Docs 

In [20]:

# 3-1
stop_words = get_stop_words()
docV3s = Parse_Docs(stop_words, "RCV1v3/")

# for k, v in docV3s.items():
#     if '80483' in k:
#         print(v.terms)
#         break


def avg_length(coll: dict):
     sum_of_length = sum(doc.doc_size for _,doc in coll.items())
     return sum_of_length / len(coll)
 
 
avg_length(docV3s)

{'suppos': 2, 'suzuki': 13, 'motor': 3, 'corp': 2, 'answer': 2, 'those': 1, 'adventuresom': 1, 'hipster': 1, 'pursu': 1, 'call': 1, 'extrem': 2, 'sport': 4, 'skyboard': 1, 'women': 1, 'box': 1, 'onli': 2, 'thing': 1, 'dismal': 1, 'sale': 6, 'record': 1, 'first': 1, 'eight': 1, 'month': 2, 'sold': 1, 'stubbi': 1, 'top': 1, 'hybrid': 1, 'between': 1, 'two': 4, 'seat': 2, 'car': 3, 'util': 3, 'vehicl': 7, 'compani': 4, 'offici': 2, 'predict': 1, 'sell': 2, 'introduc': 2, 'year': 6, 'ago': 1, 'further': 1, 'insult': 1, 'promot': 2, 'usual': 1, 'reserv': 1, 'day': 1, 'old': 1, 'doughnut': 1, 'buy': 2, 'one': 6, 'free': 2, 'bob': 2, 'sak': 2, 'oldsmobil': 1, 'farmington': 1, 'hill': 1, 'mich': 1, 'person': 1, 'cutlass': 1, 'suprem': 1, 'receiv': 1, 'leas': 1, 'long': 1, 'small': 1, 'player': 1, 'domest': 1, 'market': 2, 'reach': 1, 'peak': 1, 'unit': 1, 'batter': 1, 'news': 1, 'consum': 1, 'group': 1, 'report': 1, 'samuri': 2, 'minisport': 1, 'prone': 1, 'tip': 1, 'over': 1, 'deni': 1, 'char

357.6666666666667

In [26]:
#3-2
import math
from q1_2 import Parse_Q
from q2_1 import df
from q1_3 import get_stop_words


def my_bm25(coll: dict, q, df):
    k1 = 1.2
    k2 = 100
    b = 0.75
    N = len(coll)

    # compute avg doc length
    avg_len = avg_length(coll)

    # Tokenize query
    query_terms = Parse_Q(q, stop_words)
    

    scores = {}

    for _, doc in coll.items():
        doc_len = doc.doc_size
        doc_tf = doc.terms
        score = 0

        for term in query_terms:
            if term not in df or df[term] == 0:
                continue  # Skip terms not found in DF dictionary

            f_i = doc_tf[term] if term in doc_tf else 0
            n_i = df[term]
            qf_i = query_terms[term]

            idf = math.log((N - n_i + 0.5) / (n_i + 0.5) + 1)  # 

            K = k1 * ((1 - b) + b * (doc_len / avg_len))
            tf_component = ((k1 + 1) * f_i) / (K + f_i) if f_i > 0 else 0
            qf_component = ((k2 + 1) * qf_i) / (k2 + qf_i)          

            score += idf * tf_component  *qf_component

        scores[doc.newsID] = score

    return scores


stop_words = get_stop_words()
docV3s = Parse_Docs(stop_words, "RCV1v3/")
dfs = df(docV3s)
my_bm25(docV3s, "US EPA ranks Geo Metro car most fuel-efficient 1997 car.",  dfs)



{'741299': 2.612564748837137,
 '780723': 0.0,
 '741309': 0.0,
 '86961': 17.48588246669507,
 '780718': 0.0,
 '783803': 1.3032544561880095,
 '80483': 2.0611668818582434,
 '809481': 0.0,
 '809495': 0.0,
 '80484': 2.0611668818582434,
 '783802': 0.0,
 '807600': 0.0,
 '80884': 2.0611668818582434,
 '86042': 17.48588246669507,
 '807606': 0.0}

In [27]:
#3-3

bm25_result = my_bm25(docV3s, "US EPA ranks Geo Metro car most fuel-efficient 1997 car.",  dfs)
sorted(bm25_result.items(), key=lambda x: x[1], reverse=True)[:5]

[('86961', 17.48588246669507),
 ('86042', 17.48588246669507),
 ('741299', 2.612564748837137),
 ('80483', 2.0611668818582434),
 ('80484', 2.0611668818582434)]